In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

import os
import cv2
from PIL import Image
import time

from helper import *
from models.enet.model import *

torch.cuda.set_device(0)
torch.cuda.get_device_name(0)

'GeForce GTX 1660 Ti'

In [10]:


alpha = 0.6
# mean = [0.28689554, 0.32513303, 0.28389177]
# std = [0.18696375, 0.19017339, 0.18720214]
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = mean, std = std)
            ])

net = ENet(num_classes = 1)
net.load_state_dict(torch.load('saved_models/final_epoch9.pt', map_location = 'cpu'))
net.cuda()
net.eval()

ENet(
  (initial_block): InitialBlock(
    (conv): Conv2d(3, 13, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (out_activation): PReLU(num_parameters=1)
  )
  (downsample1_0): DownsampleBottleneck(
    (main1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (ext1): Sequential(
      (0): Conv2d(16, 4, kernel_size=(2, 2), stride=(2, 2), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (ext2): Sequential(
      (0): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
    )
    (ext3): Sequential(
      (0): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, 

In [11]:
path = '/home/himanshu/dl/dataset/valid_sih/images/'
resize_factor = 4
for i,image in enumerate(os.listdir(path)):
    if image[-3:] == 'jpg':
        frame2 = cv2.imread(os.path.join(path,image))
        print(os.path.join(path,image))
#         frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        frame2 = cv2.resize(frame2, tuple(np.flip(np.array(frame2[:,:,1].shape))//resize_factor))
        frame_pil = Image.fromarray(frame2)
        frame_tf = transform(frame_pil)
        frame_tf = frame_tf.cuda()
        frame_tf = frame_tf.unsqueeze(0)
        out = net(frame_tf)

        out = torch.sigmoid(out)
        out = out.squeeze(1).detach().cpu()
        # t = torch.Tensor([0.5])
        # out = (out > t)
        out = out.numpy()

        temp = np.zeros(frame2.shape, np.uint8)
        temp[:, :, 1] = out[0] * 255

        segmented = cv2.addWeighted(frame2, alpha, temp, (1 - alpha), 0.0)
        # plt.imshow(cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB))
        # plt.imshow(temp)
        cv2.imwrite('/home/himanshu/dl/dataset/valid_sih/masks_predicted/'+str(i)+'.jpg',segmented)
        cv2.imwrite('/home/himanshu/dl/dataset/valid_sih/masks_predicted/bw_mask/'+str(i)+'.jpg',np.squeeze(out*255))
        
    # plt.show()
print('Done!')

/home/himanshu/dl/dataset/valid_sih/images/5.jpg
/home/himanshu/dl/dataset/valid_sih/images/1.jpg
/home/himanshu/dl/dataset/valid_sih/images/7.jpg
/home/himanshu/dl/dataset/valid_sih/images/3.jpg
/home/himanshu/dl/dataset/valid_sih/images/8.jpg
/home/himanshu/dl/dataset/valid_sih/images/6.jpg
/home/himanshu/dl/dataset/valid_sih/images/4.jpg
/home/himanshu/dl/dataset/valid_sih/images/2.jpg
Done!
